<h1 style='color: rgb(77, 229, 240); text-align: left; font-family: cursive;font-size: 20px;'>Importing Liabrary...</h1>

In [2]:
# basic modules
import numpy as np
import pandas as pd

# modules to read pdf files
from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

# ignore warning
import warnings 
warnings.simplefilter('ignore') 

# for visualization
import seaborn as sns

# moduels require for machine learning
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor

# for evaluation of model
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error

<h1 style='color: red; text-align: left; font-family: cursive;font-size: 20px;'>Major challenge --> How to read data from pdf file becuase pandas don't support pdf files</h1>

<h1 style='color: lightgreen; text-align: left; font-family: cursive;font-size: 20px;'>Solution --> pdf_reader function</h1>

<h1 style='color: rgb(77, 229, 240); text-align: left; font-family: cursive;font-size: 20px;'>Function to read 'one page' of pdf file</h1>

In [3]:
'''
### Read in PDF complicated text using Pdfminer
'''      


def pdf_text_reader(pdf_file_name, pages=None):
    if pages:
        pagenums = set(pages)
    else:
        pagenums = set()

    ## 1) Initiate the Pdf text converter and interpreter
    textOutput = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, textOutput, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    ## 2) Extract text from file using the interpreter
    infile = open(pdf_file_name, 'rb')
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)        
    infile.close()
    
    ## 3) Extract the paragraphs and close the connections
    paras = textOutput.getvalue()   
    converter.close()
    textOutput.close
    
    return paras


<h1 style='color: rgb(77, 229, 240); text-align: left; font-family: cursive;font-size: 20px;'>Function to read 'whole' pdf file</h1>

In [4]:
def pdf_reader(file_name):

    file = pdf_text_reader(file_name, pages=[0]) #start from page no 0
    page_no=1
    while True:
      another_page = pdf_text_reader(file_name, pages=[page_no])

      if(len(another_page)==0): #add pages untill pages ends
            break 
      file = file+another_page
      page_no+=1
    #converting text to lower case 
    #file = [i.lower() for i in file.split('\n') if i] #splliting context of file for better handling 
    return file #it returns list containing lines of pdf pages as strings

<h1 style='color: rgb(77, 229, 240); text-align: left; font-family: cursive;font-size: 20px;'>Loading Data...</h1>

<h1 style='color: rgb(77, 229, 240); text-align: left; font-family: cursive;font-size: 20px;'>1) Reading training and testing data</h1>

In [5]:
#training data
train = pd.read_csv('dataset/train.csv')
print("Train Data :") 
train.head(3) #printing first 3 entries


Train Data :


,CandidateID,Match Percentage
0,candidate_011,13.60
1,candidate_113,36.63
2,candidate_123,54.93


In [6]:
#testing data 
test = pd.read_csv('dataset/test.csv')  
print("Test Data :") 
test.head(3) #printing first 3 entries 

Test Data :


,CandidateID
0,candidate_014
1,candidate_098
2,candidate_075


<h1 style='color: rgb(77, 229, 240); text-align: left; font-family: cursive;font-size: 15px;'>Observation --> hence there is no missing values in train and test dataset and also from train and test folder it is confirm that it contains all 90 and 60 resumes respectively</h1>

<h1 style='color: red; text-align: left; font-family: cursive;font-size: 20px;'>Challenge --> CandidateId's are shuffled so we can't use them directly</h1>

<h1 style='color: lightgreen; text-align: left; font-family: cursive;font-size: 20px;'>Solution --> we will sort dataframe based on candidateID's </h1>

In [7]:
#sort dataframe based on candidateId
train.sort_values(by ='CandidateID', inplace =True)

#reset index
train.reset_index(inplace=True)
train.drop('index',axis=1,inplace=True)

#print sorted train dataset
train.head() 

,CandidateID,Match Percentage
0,candidate_000,13.70
1,candidate_001,40.09
2,candidate_002,48.91
3,candidate_003,36.89
4,candidate_006,44.96


In [8]:
#sort dataframe based on candidateId
test.sort_values(by ='CandidateID', inplace =True)

#reset index
test.reset_index(inplace=True)
test.drop('index',axis=1,inplace=True)

#print sorted test dataset
test.head() 

,CandidateID
0,candidate_004
1,candidate_005
2,candidate_014
3,candidate_016
4,candidate_017


<h1 style='color: red; text-align: left; font-family: cursive;font-size: 20px;'>Challenge --> Cv's are in pdf format and but we need them in dataframe</h1>

<h1 style='color: lightgreen; text-align: left; font-family: cursive;font-size: 20px;'>Solution --> Loop through Cv's read them by using pdf_reader function and add to dataframe</h1>

<h1 style='color: rgb(77, 229, 240); text-align: left; font-family: cursive;font-size: 20px;'>2) Reading CV's</h1>

In [9]:
train["CV"] = 0
for i in range(90):
    train["CV"][i] = pdf_reader("dataset/trainResumes/"+str(train["CandidateID"][i])+".pdf")

train.head(3)     

,CandidateID,Match Percentage,CV
0,candidate_000,13.70,JACOB SMITH\n\nF R E S H E R\n\nPERSONAL PROFI...
1,candidate_001,40.09,Brianna Williams\n\nJ UNI OR DEVELOPER\n\nWor...
2,candidate_002,48.91,MASON QUADRADO\n\nASSOCIATE ANALYST\n\nABOUT\n...


In [10]:
test["CV"] = 0
for i in range(60):
    test["CV"][i] = pdf_reader("dataset/testResumes/"+str(test["CandidateID"][i])+".pdf")

test.head(3)    

,CandidateID,CV
0,candidate_004,OLIVIA SANTOS\n\nCONSULTANT ANALYST\n\nEXECUTI...
1,candidate_005,ARMIN FITZGERALD\n\nD A T A M A N ...
2,candidate_014,GRACE BAILRY\n\nM A C H I N E L E A R N I N ...


<h1 style='color: rgb(77, 229, 240); text-align: left; font-family: cursive;font-size: 20px;'>make new train and test sets</h1>

In [44]:
# train1 for approach1
train1=train 
train1.head()
# test1 for approach1
test1=test 
test1.head()
#remove spaces \n as well as convert it to lower case
for i in range(90):
    train1["CV"][i]=train1["CV"][i].replace(" ","")
    train1["CV"][i]=train1["CV"][i].replace("\n","")
    train1["CV"][i]=train1["CV"][i].lower()

train1.head()
#remove spaces \n as well as convert it to lower case
for i in range(60):
    test1["CV"][i]=test1["CV"][i].replace(" ","")
    test1["CV"][i]=test1["CV"][i].replace("\n","")
    test1["CV"][i]=test1["CV"][i].lower()

test1.head()

,CandidateID,CV,machinelearning,ml,artificialintelligence,ai,predict,model,algorithm,bachelor,...,facebook,complex,sap,c#,c,frontend,backend,fullstack,app,web
0,candidate_004,oliviasantosconsultantanalystexecutivesummaryd...,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,candidate_005,arminfitzgeralddatamanagerprofessionalprofilee...,1,0,1,0,1,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2,candidate_014,gracebailrymachinelearningdevelopmentanddeploy...,1,1,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
3,candidate_016,ameliamartindatasciencemajorworkexperiencenaac...,1,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
4,candidate_017,jonathanhilljuniormachinelearningengineerskill...,1,0,1,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0


<h1 style='color: rgb(77, 229, 240); text-align: left; font-family: cursive;font-size: 20px;'>Let's make list of keywords</h1>

In [45]:
keywords = ["machinelearning","ml","artificialintelligence","ai","predict","model","algorithm","bachelor","master","degree","year","experience","analy","solution","cluster","regression","classification","librar","sklearn","scikit","numpy","pandas","matplot","seaborn","deeplearn","framework","tensorflow","keras","pytorch","neuralnetwork","cnn","rnn","gans","naturallanguageprocessing","nlp","nltk","spacy","beutifulsoup","pyspark","hadoop","bigdata","datascience","data","featureengineering","model","evaluation","computervision","transformer","advance","technique","objectdetection","ocr","opencv","django","database","mangodb","sql","graphql","firebase","aws","azure","googlecloud","project","work","c++","java","python","r","develop","imageproce","tableau","tech","medal","award","matlab","research","php","b.","m.","solution","imagedetec","docker","kubernet","randomforest","decisiontree","boost","encod","stem","graduate","software","intern","excel","business","powerbi","datastructure","iot","supervised","unsupervised","reinforcement","iit","contrib","flask","sap","c#","c","frontend","backend","fullstack","app","web"]


In [46]:
len(keywords) 

110

In [47]:
for k in keywords:
    train1[k]=0
    test1[k]=0

<h1 style='color: rgb(77, 229, 240); text-align: left; font-family: cursive;font-size: 20px;'>Check for keywords and allocate score</h1>

In [48]:
for i in range(90):
    for k in keywords:
        if k in train1["CV"][i]:
            train1[k][i]=1


train1.head()

,CandidateID,Match Percentage,CV,machinelearning,ml,artificialintelligence,ai,predict,model,algorithm,...,facebook,complex,sap,c#,c,frontend,backend,fullstack,app,web
0,candidate_000,13.70,jacobsmithfresherpersonalprofileiamactivelysee...,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,candidate_001,40.09,briannawilliamsjuniordeveloperworkexperiencekp...,1,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,candidate_002,48.91,masonquadradoassociateanalystaboutcertifieddat...,1,0,0,0,1,1,0,...,0,0,0,0,1,0,0,0,0,0
3,candidate_003,36.89,rodneymichaelsassociatesoftwareengineerskillsp...,1,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,1,1
4,candidate_006,44.96,jenniferarmstrongprofilefreshercomputervisiona...,1,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0


<h1 style='color: rgb(77, 229, 240); text-align: left; font-family: cursive;font-size: 20px;'>same for test</h1>

In [49]:
for i in range(60):
    for k in keywords:
        if k in test1["CV"][i]:
            test1[k][i]=1

test.head()

,CandidateID,CV,machinelearning,ml,artificialintelligence,ai,predict,model,algorithm,bachelor,...,facebook,complex,sap,c#,c,frontend,backend,fullstack,app,web
0,candidate_004,oliviasantosconsultantanalystexecutivesummaryd...,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,candidate_005,arminfitzgeralddatamanagerprofessionalprofilee...,1,0,1,0,1,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2,candidate_014,gracebailrymachinelearningdevelopmentanddeploy...,1,1,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
3,candidate_016,ameliamartindatasciencemajorworkexperiencenaac...,1,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
4,candidate_017,jonathanhilljuniormachinelearningengineerskill...,1,0,1,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0


<h1 style='color: rgb(77, 229, 240); text-align: center; font-family: cursive;font-size: 20px;'>Training Phase</h1>

<h1 style='color: rgb(77, 229, 240); text-align: left; font-family: cursive;font-size: 20px;'>Preparaion for training and testing phase</h1>

In [50]:
xtrain = np.array(train1[keywords])
ytrain =  np.array(train1["Match Percentage"])
xtest =  np.array(test1[keywords])

<h1 style='color: rgb(77, 229, 240); text-align: left; font-family: cursive;font-size: 20px;'>3) Random Forest = 71.66</h1>

In [51]:
rf =RandomForestRegressor(n_estimators=1000)
rf.fit(xtrain,ytrain)
preR=rf.predict(xtest)

res={
    "CandidateID":list(test1.CandidateID),
    "Match Percentage":preR
}

res=pd.DataFrame(res)

res.to_csv("res.csv",index=False)